# Netflix - Sistema de Recomendação por Similaridade de Conteúdo

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

## Carga do Dataset

O dataset contém informações sobre filmes e séries disponíveis na Netflix, incluindo título, gênero, país, data de adição, classificação etária, duração, tipo (filme ou série) e descrição.

Descrição de cada coluna:
- show_id: Identificador único do show.
- type: Tipo do show (Filme ou Série).
- title: Título do show.
- director: Diretor do show.
- cast: Elenco do show.
- country: País de origem do show.
- date_added: Data em que o show foi adicionado à Netflix.
- release_year: Ano de lançamento do show.
- rating: Classificação etária do show.
- duration: Duração do show (em minutos para filmes, número de temporadas para séries).
- listed_in: Gêneros do show.

In [17]:
# Carrega o dataset
data = pd.read_csv('../data/netflix_titles.csv')

## Pré-processamento básico

In [18]:
# Preenche valores nulos com string vazia (para campos textuais)
data['cast'] = data['cast'].fillna('')
data['country'] = data['country'].fillna('')
data['listed_in'] = data['listed_in'].fillna('')

# Junta as colunas relevantes em uma só para vetorizacao
data['conteudo'] = data['cast'] + ' ' + data['country'] + ' ' + data['listed_in']

### Vetorização TF-IDF

In [19]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(data['conteudo'])

# Calcula a similaridade do cosseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Função de recomendação por título

In [20]:
def get_recomendacoes(titulo, top_n=5):
    # Verifica se o título existe
    if titulo not in data['title'].values:
        return ["Título não encontrado"]

    # Índice do título
    idx = data[data['title'] == titulo].index[0]

    # Similaridades com todos os outros
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # ignora o próprio título

    # Índices dos recomendados
    indices = [i[0] for i in sim_scores]

    return data[['title', 'type', 'release_year', 'rating', 'listed_in']].iloc[indices].to_dict(orient='records')

### Função de recomendação por campos

In [21]:
def recomendar_por_campos(fields, top_n=5):
    # Monta string semelhante ao campo 'conteudo'
    campos = [
        fields.get('cast', ''),
        fields.get('country', ''),
        fields.get('listed_in', '')
    ]
    conteudo_novo = ' '.join(campos)

    vetor = vectorizer.transform([conteudo_novo])
    sim_scores = cosine_similarity(vetor, tfidf_matrix).flatten()
    indices = np.argsort(sim_scores)[::-1][:top_n]
    return data[['title', 'type', 'release_year']].iloc[indices].to_dict(orient='records')


### Exemplo de uso

In [22]:
exemplo = get_recomendacoes('Naruto', top_n=5)
for item in exemplo:
    print(item)

for item in recomendar_por_campos({'country': 'Japan', 'release_year': '2006', 'rating': 'TV-14', 'listed_in': 'Anime', 'duration': '9'}, top_n=5):
    print(item)

{'title': 'Naruto Shippûden the Movie: Bonds', 'type': 'Movie', 'release_year': 2008, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippuden the Movie: Blood Prison', 'type': 'Movie', 'release_year': 2011, 'rating': 'TV-14', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippuden : Blood Prison', 'type': 'Movie', 'release_year': 2011, 'rating': 'TV-14', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto the Movie: Ninja Clash in the Land of Snow', 'type': 'Movie', 'release_year': 2004, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Naruto Shippûden the Movie: The Will of Fire', 'type': 'Movie', 'release_year': 2009, 'rating': 'TV-PG', 'listed_in': 'Action & Adventure, Anime Features, International Movies'}
{'title': 'Pop Team Epic', 'type': 'TV Show', 'release_year': 2018}
{'title': 

### Exportação para uso no Flask

In [23]:
with open('../models/dados_netflix.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('../models/cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

with open('../models/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)